# Impact on Initial Mass Function

When determining stellar masses in young stellar associations, non-magnetic models are often adopted. However, if magnetic inhibition of convection is an important process in governing the structure of young pre-main-sequence stars, then the mass-Teff relationship will be shifted. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt